# Notebook for verifying the rotator prediction code for BOTH ports

### Import Modules

In [1]:

import os
import sys
from datetime import datetime
from zoneinfo import ZoneInfo

import astropy.coordinates
from astropy.time import Time
import astropy.units as u
import numpy as np
import yaml

### Define a Minimum Required Working Config

In [2]:
config = {
    "telescope": {
        "host": "thor",
        "simhost": "localhost",
        "comm_port": 8220,
        "home_alt_degs": 45.0,
        "home_az_degs": 220,
        "min_alt": 15,
        "max_alt": 85.0,
        "ports": {
            "default": 1,
            1: {
                "name": "winter",
                "rotator": {
                    "field_angle_zeropoint": 65.0,
                    "home_degs": 65.5,
                    "max_degs": 120.0,
                    "min_degs": -90.0,
                },
                "cameras": {
                    "winter": {
                        "pointing_model_file": "pointing_model_winter_20250529_downselected.pxp",
                    }
                },
            },
            2: {
                "name": "summer",
                "rotator": {
                    "field_angle_zeropoint": 155.0,
                    "home_degs": 65.0,
                    "max_degs": 225.0,
                    "min_degs": -95.0,
                },
                "cameras": {
                    "spring": {
                        "pointing_model_file": "pointing_model_spring_20251022.pxp",
                    }
                },
            },
        },
    },
    "site": {
        "lat": "33d21m21.6s",
        "lon": "-116d51m46.8s",
        "height": 1706,
        "height_units": "m",
        "timezone": "America/Los_Angeles",
    },
}

### Test Results which must be replicated


In [3]:

test_cases = {
    "spring": [
        { 
            "name": "M26",
            "ra" : "18:45:12.60",
            "dec": "-09:23:59.9",
            "obstime" : datetime(2025, 11, 10, 13, 20, 0, tzinfo=ZoneInfo(config["site"]["timezone"])),
            "actual_mech_angle": 45.0,
        },
        {
            "name": "M15",
            "ra" : "21:30:00.45",
            "dec": "12:10:00.0",
            "obstime" : datetime(2025, 11, 10, 13, 38, 0, tzinfo=ZoneInfo(config["site"]["timezone"])),
            "actual_mech_angle": 61.3,
        },

        # Add more spring test cases here
        {
            "name": "M94",
            "ra": "12:50:54.8",
            "dec": "41:06:59.9",
            "obstime": datetime(2025, 11, 10, 8, 55, 0, tzinfo=ZoneInfo(config["site"]["timezone"])),
            "actual_mech_angle": 25.0,
        },
        {
            "name": "stow position",
            "ra": "14:42:32.30",
            "dec": "57:08:56.8",
            "obstime": datetime(2025, 11, 10, 12, 58, 0, tzinfo=ZoneInfo(config["site"]["timezone"])),
            "actual_mech_angle": 85.0,
        }
    ],
    "winter": [
        {
            "name": "Winter Test 1",
            "ra": "09:27:35.49",
            "dec": "-08:39:30.2",
            "obstime": datetime(2025, 11, 10, 7, 27, 0, tzinfo=ZoneInfo(config["site"]["timezone"])),
            "actual_mech_angle": -3.8,
        },
        # Add more winter test cases here

    ],
}


## Method 1: Use the roboOperator Methods:

This approach makes a test stub which allows us to use the `get_safe_rotator_angle` method directly imported from the `RoboOperator` class, without populating the full module and avoiding the `__init__` which requires a lot of other input.

In [4]:
from wsp.control.roboOperator import RoboOperator
from astropy.coordinates import Angle, EarthLocation
from types import MethodType
from types import SimpleNamespace

results = {}
def make_robo_stub(config, camname="spring"):
    # create a RoboOperator instance without running its __init__
    robo = RoboOperator.__new__(RoboOperator)

    # required attrs your method touches
    robo.config = config

    # site (EarthLocation)
    lat = Angle(config["site"]["lat"])
    lon = Angle(config["site"]["lon"])
    height = config["site"]["height"] * u.Unit(config["site"]["height_units"])
    site = EarthLocation(lat=lat, lon=lon, height=height)
    robo.ephem = SimpleNamespace(site=site)

    # telescope.port must index config["telescope"]["ports"][port]
    # choose a key that actually exists in your config, e.g. "spring"
    port_key = camname  # or map however your config is structured
    robo.telescope = SimpleNamespace(port=port_key)

    # the method calls this predicate; provide a simple one
    def is_rotator_mech_angle_possible(self, predicted_rotator_mechangle, rotator_min_degs, rotator_max_degs):
        return rotator_min_degs <= predicted_rotator_mechangle <= rotator_max_degs

    robo.is_rotator_mech_angle_possible = MethodType(is_rotator_mech_angle_possible, robo)

    # optional: a helper if you want to switch cameras by name -> port
    def switchCamera(self, name):
        self.camname = name
        if name == "winter":
            self.telescope.port = 1
        elif name == "spring":
            self.telescope.port = 2
        else:  
            self.telescope.port = 0  # or raise an error
        print(f"Switched to {name} camera on port {self.telescope.port}")
    robo.switchCamera = MethodType(switchCamera, robo)

    # set initial camera/port
    robo.switchCamera(camname)

    return robo

"""

fa, ma = RoboOperator.get_safe_rotator_angle(
        robo,
        ra_hours=target_ra_j2000_hours,
        dec_deg=target_dec_j2000_deg,
        target_field_angle=target_field_angle,
        obstime=obstime,
        verbose = True
    )
"""



# Run tests
results = {}
threshold = 2.0  # degs

for camname in ["winter", "spring"]:
    # create the robo stub
    robo = make_robo_stub(config, camname=camname)
    
    print("\n" + "="*60)
    print(f"CAMERA: {camname.upper()} (Port {robo.telescope.port})")
    print("="*60)
    
    if camname not in results:
        results[camname] = []
    
    for test_case in test_cases[camname]:
        print(f"\n--- {test_case['name']} ---")
        
        ra = test_case["ra"]
        dec = test_case["dec"]
        obstime_datetime = test_case["obstime"]
        actual_mech_angle = test_case["actual_mech_angle"]
        
        # Convert to UTC
        utc_obstime_datetime = obstime_datetime.astimezone(ZoneInfo("UTC"))
        print(f"Obstime: Local: {obstime_datetime} --> UTC: {utc_obstime_datetime}")
        obstime = Time(utc_obstime_datetime, location=robo.ephem.site)
        print(f"Obstime UTC: {obstime}. MJD = {obstime.mjd:.1f}")
        
        # Calculate the J2000 coords
        ra_hours = astropy.coordinates.Angle(ra, unit=u.hour)
        dec_deg = astropy.coordinates.Angle(dec, unit=u.deg)
        j2000_coords = astropy.coordinates.SkyCoord(ra_hours, dec_deg, frame="icrs")
        
        target_ra_j2000_hours = j2000_coords.ra.hour
        target_dec_j2000_deg = j2000_coords.dec.deg
        target_field_angle = config["telescope"]["ports"][robo.telescope.port]["rotator"]["field_angle_zeropoint"]
        
        print(f"J2000 target coords: RA = {target_ra_j2000_hours:0.5f} hours, DEC = {target_dec_j2000_deg:0.5f} degs")
        print(f"Target field angle: {target_field_angle} degs")
        
        # Calculate safe rotator angle
        fa, ma = RoboOperator.get_safe_rotator_angle(
            robo,
            ra_hours=target_ra_j2000_hours,
            dec_deg=target_dec_j2000_deg,
            target_field_angle=target_field_angle,
            obstime=obstime,
            verbose=True
        )
        
        # Check against actual
        print(f"Calculated Mech Angle: {ma:0.3f} degs, Actual Mech Angle: {actual_mech_angle} degs")
        angle_error = np.mod(abs(ma - actual_mech_angle), 360.0)
        print(f"Angle error: {angle_error:0.3f} degs")
        
        if angle_error < threshold:
            print(f"✓ PASS: Prediction within {threshold} deg threshold")
            passed = True
        else:
            print(f"✗ FAIL: Prediction outside {threshold} deg threshold")
            passed = False
        
        results[camname].append({
            "name": test_case["name"],
            "passed": passed,
            "error": angle_error
        })

# Print summary
print("\n" + "="*60)
print("SUMMARY OF ROTATOR PREDICTION TESTS")
print("="*60)

for camname in ["spring", "winter"]:
    print(f"\n{camname.upper()}:")
    total_tests = len(results[camname])
    passed_tests = sum(1 for r in results[camname] if r["passed"])
    
    for result in results[camname]:
        status = "✓ PASS" if result["passed"] else "✗ FAIL"
        print(f"  {status}: {result['name']} (error: {result['error']:.3f} deg)")
    
    print(f"  --> {passed_tests}/{total_tests} tests passed")

total_all = sum(len(results[cam]) for cam in results)
passed_all = sum(sum(1 for r in results[cam] if r["passed"]) for cam in results)
print(f"\nOVERALL: {passed_all}/{total_all} tests passed")

CalTracker: wsp_path = /Users/nlourie/Desktop/Work/MIT/WINTER/GIT/observatory/wsp
roboManager: wsp_path = /Users/nlourie/Desktop/Work/MIT/WINTER/GIT/observatory/wsp
FocusTracker: wsp_path = /Users/nlourie/Desktop/Work/MIT/WINTER/GIT/observatory/wsp
focusing: wsp_path = /Users/nlourie/Desktop/Work/MIT/WINTER/GIT/observatory/wsp
wsp_path = /Users/nlourie/Desktop/Work/MIT/WINTER/GIT/observatory/wsp
genstats: wsp_path = /Users/nlourie/Desktop/Work/MIT/WINTER/GIT/observatory/wsp
data_handler: wsp_path = /Users/nlourie/Desktop/Work/MIT/WINTER/GIT/observatory/wsp
control: wsp_path = /Users/nlourie/Desktop/Work/MIT/WINTER/GIT/observatory/wsp
Switched to winter camera on port 1

CAMERA: WINTER (Port 1)

--- Winter Test 1 ---
Obstime: Local: 2025-11-10 07:27:00-08:00 --> UTC: 2025-11-10 15:27:00+00:00
Obstime UTC: 2025-11-10 15:27:00. MJD = 60989.6
J2000 target coords: RA = 9.45986 hours, DEC = -8.65839 degs
Target field angle: 65.0 degs


/Users/nlourie/Desktop/Work/MIT/WINTER/GIT/observatory/.conda/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.20) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


target_field_angle = 65.000 degs
parallactic angle = 26.366 degs
target alt = 42.662 degs
target az = 211.261 degs

##########################################
No rotator wrap predicted
Adjusted field angle --> 65.0
New target mech angle = -4.027364090602049
##########################################
Calculated Mech Angle: -4.027 degs, Actual Mech Angle: -3.8 degs
Angle error: 0.227 degs
✓ PASS: Prediction within 2.0 deg threshold
Switched to spring camera on port 2

CAMERA: SPRING (Port 2)

--- M26 ---
Obstime: Local: 2025-11-10 13:20:00-08:00 --> UTC: 2025-11-10 21:20:00+00:00
Obstime UTC: 2025-11-10 21:20:00. MJD = 60989.9
J2000 target coords: RA = 18.75350 hours, DEC = -9.39997 degs
Target field angle: 155.0 degs
telescope port 2, alt sign = +1
target_field_angle = 155.000 degs
parallactic angle = -30.503 degs
target alt = 39.533 degs
target az = 142.759 degs

##########################################
Rotator wrapping < min, adjusting by -180 deg.
Adjusted field angle --> -25.0
New

## Method 2: Directly Define the Rotator Calculation
This block is useful for debugging/directly editing the functions without needing to messa round with the code in roboOperator.

In [5]:
from wsp.control.roboOperator import RoboOperator
from astropy.coordinates import Angle, EarthLocation
from types import MethodType
from types import SimpleNamespace


def is_rotator_mech_angle_possible(
    predicted_rotator_mechangle, rotator_min_degs, rotator_max_degs
):
    return (predicted_rotator_mechangle > rotator_min_degs) and (
        predicted_rotator_mechangle < rotator_max_degs
    )

def get_safe_rotator_angle(
    ra_hours,
    dec_deg,
    target_field_angle,
    # not in original
    site,
    config,
    telescope_port,
    ############
    obstime=None,
    verbose=False,
    
):
    """
    takes in the target field angle, and then returns a field angle and
    mechanical angle pair that corresponds to the best safe rotator
    position within the allowed cable wrap range. Evaluates these 5
    choices (ranked best to worst):
        1. target field angle
        2. target field angle - 360 deg
        3. target field angle + 360 deg
        4. target field angle - 180 deg
        5. target field angle + 180 deg

    """

    if obstime is None:
        obstime = astropy.time.Time(datetime.utcnow(), location=self.ephem.site)

    self_target_ra_j2000_hours = ra_hours
    self_target_dec_j2000_deg = dec_deg

    j2000_ra = self_target_ra_j2000_hours * u.hour
    j2000_dec = self_target_dec_j2000_deg * u.deg
    j2000_coords = astropy.coordinates.SkyCoord(
        ra=j2000_ra, dec=j2000_dec, frame="icrs"
    )

    ra_deg = j2000_coords.ra.deg

    # lat = astropy.coordinates.Angle(self.config['site']['lat'])
    # lon = astropy.coordinates.Angle(self.config['site']['lon'])
    # height = self.config['site']['height'] * u.Unit(self.config['site']['height_units'])
    if site is None:
        site = (
            self.ephem.site
        )  # astropy.coordinates.EarthLocation(lat = lat, lon = lon, height = height)
    frame = astropy.coordinates.AltAz(obstime=obstime, location=site)
    local_coords = j2000_coords.transform_to(frame)
    self_target_alt = local_coords.alt.deg
    self_target_az = local_coords.az.deg

    dec = dec_deg * np.pi / 180.0
    lst = obstime.sidereal_time("mean").rad
    hour_angle = lst - ra_deg * np.pi / 180.0
    if hour_angle < -1 * np.pi:
        hour_angle += 2 * np.pi
    if hour_angle > np.pi:
        hour_angle -= 2 * np.pi

    lat = astropy.coordinates.Angle(config["site"]["lat"]).rad

    parallactic_angle = (
        np.arctan2(
            np.sin(hour_angle),
            np.tan(lat) * np.cos(dec) - np.sin(dec) * np.cos(hour_angle),
        )
        * 180
        / np.pi
    )

    possible_target_field_angles = [
        target_field_angle,
        target_field_angle - 360.0,
        target_field_angle + 360.0,
        target_field_angle - 180.0,
        target_field_angle + 180.0,
    ]
    """
    # This is the baseline which works for port 1 but not port 2:
    possible_target_mech_angles = [
        (target_field_angle - parallactic_angle - self_target_alt)
        for target_field_angle in possible_target_field_angles
    ]
    """

    alt_sign = -1
    if str(telescope_port) in ("2", "spring", "SPRING"):
        alt_sign = +1

    # Compute candidate mech angles with the port-dependent sign
    possible_target_mech_angles = [
        (fa - parallactic_angle + alt_sign * self_target_alt)
        for fa in possible_target_field_angles
    ]


    print(f"target_field_angle = {target_field_angle:0.3f} degs")
    print(f"parallactic angle = {parallactic_angle:0.3f} degs")
    print(f"target alt = {self_target_alt:0.3f} degs")
    print(f"target az = {self_target_az:0.3f} degs")
    

    messages = [
        "No rotator wrap predicted",
        "Rotator wrapping < min, adjusting by -360 deg.",
        "Rotator wrapping > max, adjusting by +360 deg.",
        "Rotator wrapping < min, adjusting by -180 deg.",
        "Rotator wrapping > max, adjusting by +180 deg.",
    ]

    if verbose:
        print("\n##########################################")
    for ind, possible_target_mech_angle in enumerate(possible_target_mech_angles):
        if is_rotator_mech_angle_possible(
            predicted_rotator_mechangle=possible_target_mech_angle,
            rotator_min_degs=config["telescope"]["ports"][telescope_port][
                "rotator"
            ]["min_degs"],
            rotator_max_degs=config["telescope"]["ports"][telescope_port][
                "rotator"
            ]["max_degs"],
        ):
            self_target_mech_angle = possible_target_mech_angle
            self_target_field_angle = possible_target_field_angles[ind]
            if verbose:
                print(messages[ind])
                print(f"Adjusted field angle --> {self_target_field_angle}")
                print(f"New target mech angle = {self_target_mech_angle}")
            break
    if verbose:
        print("##########################################")

    return self_target_field_angle, self_target_mech_angle

def make_robo_stub(config, camname="spring"):
    # create a RoboOperator instance without running its __init__
    robo = RoboOperator.__new__(RoboOperator)

    # required attrs your method touches
    robo.config = config

    # site (EarthLocation)
    lat = Angle(config["site"]["lat"])
    lon = Angle(config["site"]["lon"])
    height = config["site"]["height"] * u.Unit(config["site"]["height_units"])
    site = EarthLocation(lat=lat, lon=lon, height=height)
    robo.ephem = SimpleNamespace(site=site)

    # telescope.port must index config["telescope"]["ports"][port]
    port_key = camname
    robo.telescope = SimpleNamespace(port=port_key)

    # the method calls this predicate; provide a simple one
    def is_rotator_mech_angle_possible(self, predicted_rotator_mechangle, rotator_min_degs, rotator_max_degs):
        return rotator_min_degs <= predicted_rotator_mechangle <= rotator_max_degs

    robo.is_rotator_mech_angle_possible = MethodType(is_rotator_mech_angle_possible, robo)

    # optional: a helper if you want to switch cameras by name -> port
    def switchCamera(self, name):
        self.camname = name
        if name == "winter":
            self.telescope.port = 1
        elif name == "spring":
            self.telescope.port = 2
        else:  
            self.telescope.port = 0  # or raise an error
        print(f"Switched to {name} camera on port {self.telescope.port}")
    robo.switchCamera = MethodType(switchCamera, robo)

    # set initial camera/port
    robo.switchCamera(camname)

    return robo


# Define test cases for each camera


"""
# this one is wrong??
{
    "name": "alphard",
    "ra": "06:19:06.71",
    "dec": "+34:40:00.4",
    "obstime": datetime(2025, 11, 10, 8, 12, 0, tzinfo=ZoneInfo(config["site"]["timezone"])),
    "actual_mech_angle": 155.0,
},
"""
    

test_cases = {
    "spring": [
        { 
            "name": "M26",
            "ra" : "18:45:12.60",
            "dec": "-09:23:59.9",
            "obstime" : datetime(2025, 11, 10, 13, 20, 0, tzinfo=ZoneInfo(config["site"]["timezone"])),
            "actual_mech_angle": 45.0,
        },
        {
            "name": "M15",
            "ra" : "21:30:00.45",
            "dec": "12:10:00.0",
            "obstime" : datetime(2025, 11, 10, 13, 38, 0, tzinfo=ZoneInfo(config["site"]["timezone"])),
            "actual_mech_angle": 61.3,
        },

        # Add more spring test cases here
        {
            "name": "M94",
            "ra": "12:50:54.8",
            "dec": "41:06:59.9",
            "obstime": datetime(2025, 11, 10, 8, 55, 0, tzinfo=ZoneInfo(config["site"]["timezone"])),
            "actual_mech_angle": 25.0,
        },
        {
            "name": "stow position",
            "ra": "14:42:32.30",
            "dec": "57:08:56.8",
            "obstime": datetime(2025, 11, 10, 12, 58, 0, tzinfo=ZoneInfo(config["site"]["timezone"])),
            "actual_mech_angle": 85.0,
        }
    ],
    "winter": [
        {
            "name": "Winter Test 1",
            "ra": "09:27:35.49",
            "dec": "-08:39:30.2",
            "obstime": datetime(2025, 11, 10, 7, 27, 0, tzinfo=ZoneInfo(config["site"]["timezone"])),
            "actual_mech_angle": -3.8,
        },
        # Add more winter test cases here

    ],
}

# Run tests
results = {}
threshold = 2.0  # degs

for camname in ["winter", "spring"]:
    # create the robo stub
    robo = make_robo_stub(config, camname=camname)
    
    print("\n" + "="*60)
    print(f"CAMERA: {camname.upper()} (Port {robo.telescope.port})")
    print("="*60)
    
    if camname not in results:
        results[camname] = []
    
    for test_case in test_cases[camname]:
        print(f"\n--- {test_case['name']} ---")
        
        ra = test_case["ra"]
        dec = test_case["dec"]
        obstime_datetime = test_case["obstime"]
        actual_mech_angle = test_case["actual_mech_angle"]
        
        # Convert to UTC
        utc_obstime_datetime = obstime_datetime.astimezone(ZoneInfo("UTC"))
        print(f"Obstime: Local: {obstime_datetime} --> UTC: {utc_obstime_datetime}")
        obstime = Time(utc_obstime_datetime, location=robo.ephem.site)
        print(f"Obstime UTC: {obstime}. MJD = {obstime.mjd:.1f}")
        
        # Calculate the J2000 coords
        ra_hours = astropy.coordinates.Angle(ra, unit=u.hour)
        dec_deg = astropy.coordinates.Angle(dec, unit=u.deg)
        j2000_coords = astropy.coordinates.SkyCoord(ra_hours, dec_deg, frame="icrs")
        
        target_ra_j2000_hours = j2000_coords.ra.hour
        target_dec_j2000_deg = j2000_coords.dec.deg
        target_field_angle = config["telescope"]["ports"][robo.telescope.port]["rotator"]["field_angle_zeropoint"]
        
        print(f"J2000 target coords: RA = {target_ra_j2000_hours:0.5f} hours, DEC = {target_dec_j2000_deg:0.5f} degs")
        print(f"Target field angle: {target_field_angle} degs")
        
        # Calculate safe rotator angle
        fa, ma = get_safe_rotator_angle(
            ra_hours=target_ra_j2000_hours,
            dec_deg=target_dec_j2000_deg,
            target_field_angle=target_field_angle,
            site=robo.ephem.site,
            config=robo.config,
            telescope_port=robo.telescope.port,
            obstime=obstime,
            verbose=True
        )
        
        # Check against actual
        print(f"Calculated Mech Angle: {ma:0.3f} degs, Actual Mech Angle: {actual_mech_angle} degs")
        angle_error = np.mod(abs(ma - actual_mech_angle), 360.0)
        print(f"Angle error: {angle_error:0.3f} degs")
        
        if angle_error < threshold:
            print(f"✓ PASS: Prediction within {threshold} deg threshold")
            passed = True
        else:
            print(f"✗ FAIL: Prediction outside {threshold} deg threshold")
            passed = False
        
        results[camname].append({
            "name": test_case["name"],
            "passed": passed,
            "error": angle_error
        })

# Print summary
print("\n" + "="*60)
print("SUMMARY OF ROTATOR PREDICTION TESTS")
print("="*60)

for camname in ["spring", "winter"]:
    print(f"\n{camname.upper()}:")
    total_tests = len(results[camname])
    passed_tests = sum(1 for r in results[camname] if r["passed"])
    
    for result in results[camname]:
        status = "✓ PASS" if result["passed"] else "✗ FAIL"
        print(f"  {status}: {result['name']} (error: {result['error']:.3f} deg)")
    
    print(f"  --> {passed_tests}/{total_tests} tests passed")

total_all = sum(len(results[cam]) for cam in results)
passed_all = sum(sum(1 for r in results[cam] if r["passed"]) for cam in results)
print(f"\nOVERALL: {passed_all}/{total_all} tests passed")

Switched to winter camera on port 1

CAMERA: WINTER (Port 1)

--- Winter Test 1 ---
Obstime: Local: 2025-11-10 07:27:00-08:00 --> UTC: 2025-11-10 15:27:00+00:00
Obstime UTC: 2025-11-10 15:27:00. MJD = 60989.6
J2000 target coords: RA = 9.45986 hours, DEC = -8.65839 degs
Target field angle: 65.0 degs
target_field_angle = 65.000 degs
parallactic angle = 26.366 degs
target alt = 42.662 degs
target az = 211.261 degs

##########################################
No rotator wrap predicted
Adjusted field angle --> 65.0
New target mech angle = -4.027364090602049
##########################################
Calculated Mech Angle: -4.027 degs, Actual Mech Angle: -3.8 degs
Angle error: 0.227 degs
✓ PASS: Prediction within 2.0 deg threshold
Switched to spring camera on port 2

CAMERA: SPRING (Port 2)

--- M26 ---
Obstime: Local: 2025-11-10 13:20:00-08:00 --> UTC: 2025-11-10 21:20:00+00:00
Obstime UTC: 2025-11-10 21:20:00. MJD = 60989.9
J2000 target coords: RA = 18.75350 hours, DEC = -9.39997 degs
Targe